# Temperature Scaling Calibration

This notebook finds the optimal temperature parameter (tau) that minimizes ECE on the validation set.
The optimal tau is saved to `temperature.json` in the checkpoint directory for use during inference.

In [0]:
%pip install tqdm

In [0]:
import os
import sys

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

print(f"Current working directory: {os.getcwd()}")

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

In [0]:
import torch
from src.training.calibration import calibrate_from_split_csv

In [0]:
# === Configuration ===

# Path to the trained model checkpoint
checkpoint_path = "/Volumes/main_dev/dld_ml_anticheat_test/anticheat_test_volume/pgc_wwcd/pgc_results/checkpoints/run22_emb1024_head2_layer6_drop0.1_lr1e-4_weighted_cox_v11/best.pt"

# Path to data folder
folder_path = "/Volumes/main_dev/dld_ml_anticheat_test/anticheat_test_volume/pgc_wwcd/pgc_features/inference_v2"

# Path to split CSV file
split_csv_path = "/Volumes/main_dev/dld_ml_anticheat_test/anticheat_test_volume/pgc_wwcd/pgc_features/inference_v2/split_files.csv"

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [0]:
# Check file existence
if not os.path.exists(checkpoint_path):
    print(f"[Warning] Checkpoint not found: {checkpoint_path}")
else:
    print(f"[OK] Checkpoint found: {checkpoint_path}")

if not os.path.exists(split_csv_path):
    print(f"[Warning] Split CSV not found: {split_csv_path}")
else:
    print(f"[OK] Split CSV found: {split_csv_path}")

In [0]:
# Run calibration
if os.path.exists(checkpoint_path) and os.path.exists(split_csv_path):
    results = calibrate_from_split_csv(
        checkpoint_path=checkpoint_path,
        folder_path=folder_path,
        split_csv_path=split_csv_path,
        device=device,
        batch_size=64,
        num_workers=4,
        save_tau=True,
    )
else:
    print("Skipping calibration due to missing files.")

In [0]:
# Display results
if "results" in dir():
    print("\n" + "=" * 50)
    print("Calibration Summary")
    print("=" * 50)
    print(f"Optimal Temperature (tau): {results['optimal_tau']:.4f}")
    print(f"ECE before calibration:    {results['ece_without_tau']:.6f}")
    print(f"ECE after calibration:     {results['ece_with_tau']:.6f}")
    improvement = (results['ece_without_tau'] - results['ece_with_tau']) / results['ece_without_tau'] * 100
    print(f"ECE improvement:           {improvement:.2f}%")